# Data Preprocessing

對新聞、討論的文章進行資料前處理，包含斷詞、去除標點符號、計算 tf-idf、特徵選取等等，以利後續的模型訓練。

## 使用 CKIP Tagger 進行斷詞

In [26]:
# 先把我們需要的函數載入
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./data_ckip") # 斷詞
pos = POS("./data_ckip") # 詞性標注
ner = NER("./data_ckip") # 命名實體識別 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2024-04-13 16:15:52.142556: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_c

In [14]:
# 載入資料集
import pandas as pd
data_dir = '../data/'

df_news = pd.read_csv(data_dir + 'news_filtered_labeled.csv')

In [15]:
df_chip = pd.read_csv(data_dir + '籌碼數據-2年_by_date.csv')
df_chip.head()

,date,foreign_investor_surplus,investment_trust_surplus,dealer_surplus
0,2022-03-01 00:00:00,40895704,6247000,13420418
1,2022-03-02 00:00:00,-58004507,5013000,8992716
2,2022-03-03 00:00:00,45348028,3661000,25176165
3,2022-03-04 00:00:00,-865155,14308000,-5958158
4,2022-03-07 00:00:00,-62514759,2651000,-21795347


In [19]:
# merge news and chip data by date
df_news['date'] = pd.to_datetime(df_news['date'])
df_chip['date'] = pd.to_datetime(df_chip['date'])
# merge the data by date in df_chip and post_time in df_news
df = pd.merge(df_news, df_chip, how='inner', on='date')
# drop column 'Unnamed: 0'
df = df.drop(columns=['Unnamed: 0'])

In [20]:
# Save the merged data
df.to_csv(data_dir + 'news_filtered_merged.csv', index=False)

### 移除被標註為「持平」的文章

In [3]:
days = 3

# 把標籤為 -1 的 row 全部移除
df_news = df_news[df_news['label_day' + str(days)] != -1]
Y = get_label(df_news, days) # 取得標籤

# 提取正文內容
corpus = df_news['content'].tolist()

In [4]:
# 進行斷詞
word_sentence_list = ws(corpus, 
                        sentence_segmentation=True,
                        segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、', ' ', '.'})
# 移除標點符號
punc = ['，', '。', '、', '：', '；', '？', '！', '「', '」', '（', '）', '『', '』', '—', '－', '～', '…', '‧', '《', '》', '〈', '〉', '﹏﹏']
eng_punc = [',', '.', ':', ';', '?', '!', '(', ')', '[', ']', '&', '@', '#', '$', '%', '-', '_', '*', '/', '\\', '+', '=', '>', '<', '"', "'", '’', '‘', '“', '”', ' ']
stop_words = ['全文', '日', '月', '年', 'br', '中央社', '公司', '上午', '下午', '日期'] # 停用詞

# 只要詞裡面含有數字就移除
word_sentence_list = [[word for word in sentence if not any(char.isdigit() for char in word)] for sentence in word_sentence_list]
word_sentence_list = [[word for word in sentence if word not in punc] for sentence in word_sentence_list]
word_sentence_list = [[word for word in sentence if word not in eng_punc] for sentence in word_sentence_list]
word_sentence_list = [[word for word in sentence if word not in stop_words] for sentence in word_sentence_list]

## 取得 n-gram (n = 1 ~ 3) 的 BOW Count
最後會被存在名為 `bow_count` 的 scipy.sparse._csr.csr_matrix 物件中。

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(ngram_range=(1, 3))
bow_count = bow.fit_transform([' '.join(sentence) for sentence in word_sentence_list])
print(bow_count.shape)

(5531, 923189)


## 取得 n-gram (n = 1 ~ 3) 的 tf-idf
最後會被存在名為 `tfidf` 的 scipy.sparse._csr.csr_matrix 物件中。

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tv = TfidfVectorizer(ngram_range=(1, 3))
tfidf = tv.fit_transform([' '.join(sentence) for sentence in word_sentence_list])
print(tfidf.shape)

(5531, 923189)


## Chi-square 特徵選取
根據標籤內容，計算每個詞的 chi-square 值，並選取前 1000 個詞作為特徵。

In [7]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np


# 選取前 1000 個特徵
ch2 = SelectKBest(chi2, k=1000)
X_chi2 = ch2.fit_transform(tfidf, Y)
print(X_chi2.shape)

(5531, 1000)


In [8]:
# 顯示 X_chi2 的特徵名稱
feature_names = bow.get_feature_names_out()
mask = ch2.get_support() # 這個 mask 會告訴你哪些特徵被選取了
selected_features = feature_names[mask]
# 從 selected features 中隨機選取 10 個特徵
# 取出 chi2 分數最高的 100 個特徵
top_100 = np.argsort(ch2.scores_)[-100:]
selected_features = feature_names[top_100]
print(selected_features)

['出席' '電芯' '早餐' '中信 投顧' '長榮 國際 單位' '台塑 智能' '私募' '上市 中櫃 單位' '中櫃 單位' '上市 中櫃'
 '換約 漲幅' 'jtb' '瑞銀' '張紹豐' '鮮食' '上市 新興' '新興 單位' '上市 新興 單位' '發行 長榮 國際' '火箭'
 '減資案' '取消' '速報 中航' '盤中 速報 中航' '彭士孝' '旅展' '現金 減資' '先生' '鄭文燦' '年度 分派 股利'
 '年度 分派' '買回' '協商' '減資' '中航' '修訂案' '外站' '參加' '證券 舉辦 法說會' '江耀宗' '台股 企業日'
 '企業日' '義大利麵' '元富 證券 舉辦' '長家' '衰退率' '買賣 br' '列車' '參加 元富' '參加 元富 證券' '副駕駛'
 'br 盤中 速報' 'br 盤中' '配送' '全家' '盤中 速報' '租回' '爭議' '上市 亞航' '上市 亞航 單位' '亞航 單位'
 '台灣 高鐵' '閱讀 時報 台北' '太空' '時報 資訊' '速報' '閱讀 時報' '大榮' '嘉里 大榮' '工會' '盈餘 閱讀 時報'
 '上市 單位' '盈餘 閱讀' 'br' '罷工' '上市 裕民' '投控 單位' '上市 台驊' '上市 台驊 投控' '台驊 投控 單位'
 '資訊 航運股' '時報 資訊 航運股' '裕民 單位' '上市 裕民 單位' '機師 工會' '嘉里' '閱讀 上市' '年增 閱讀 上市'
 '上市' '高鐵' '顏益財' '上市 長榮' '單位' '長榮 單位' '上市 長榮 單位' '財經 公告' '營收 年增 閱讀'
 '年增 閱讀' '閱讀' '機師']


## 儲存處理好的資料

In [9]:
# Save X_chi2
from scipy import sparse
sparse.save_npz(data_dir + 'X_chi2.npz', X_chi2)
sparse.save_npz(data_dir + 'Y.npz', sparse.csr_matrix(Y))

# 函數定義

In [21]:
def word_segmentation(corpus, ws):
    # 進行斷詞
    word_sentence_list = ws(corpus, 
                            sentence_segmentation=True,
                            segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、', ' ', '.'})
    # 移除標點符號
    punc = ['，', '。', '、', '：', '；', '？', '！', '「', '」', '（', '）', '『', '』', '—', '－', '～', '…', '‧', '《', '》', '〈', '〉', '﹏﹏']
    eng_punc = [',', '.', ':', ';', '?', '!', '(', ')', '[', ']', '&', '@', '#', '$', '%', '-', '_', '*', '/', '\\', '+', '=', '>', '<', '"', "'", '’', '‘', '“', '”', ' ']
    stop_words = ['全文', '日', '月', '年', 'br', '中央社', '公司', '上午', '下午', '日期'] # 停用詞

    # 只要詞裡面含有數字就移除
    word_sentence_list = [[word for word in sentence if not any(char.isdigit() for char in word)] for sentence in word_sentence_list]
    word_sentence_list = [[word for word in sentence if word not in punc] for sentence in word_sentence_list]
    word_sentence_list = [[word for word in sentence if word not in eng_punc] for sentence in word_sentence_list]
    word_sentence_list = [[word for word in sentence if word not in stop_words] for sentence in word_sentence_list]
        
    return word_sentence_list

In [22]:
def preprocessing(df_text, Y, ws):
    corpus = df_text['content'].tolist()
    word_sentence_list = word_segmentation(corpus, ws)
    tv = TfidfVectorizer(ngram_range=(1, 3))
    tfidf = tv.fit_transform([' '.join(sentence) for sentence in word_sentence_list])
    # Chi2 feature selection
    ch2 = SelectKBest(chi2, k=1000)
    X_chi2 = ch2.fit_transform(tfidf, Y)
    # Add foreign_investor_surplus, investment_trust_surplus, and dealer_surplus to X_chi2
    X_chi2 = sparse.hstack((X_chi2, sparse.csr_matrix(df_text[['foreign_investor_surplus', 'investment_trust_surplus', 'dealer_surplus']])))
    return X_chi2

In [28]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np

df_text = pd.read_csv(data_dir + 'news_filtered_merged.csv')
Y = get_label(df_text, 3)
X = preprocessing(df_text, Y, ws)

NameError: name 'TfidfVectorizer' is not defined